In [61]:
import pandas as pd
import requests
import json
import pymysql
import os
import getpass

In [2]:
api_key = os.getenv("IBM_APIKEY")

# Translate product category names from Portuguese to English
## Using IBM Watson Translate API

Try to translate a word

In [25]:
to_translate = "hello"

In [26]:
url = "https://api.eu-de.language-translator.watson.cloud.ibm.com/instances/58781bbf-1a05-4c1d-a4ee-338e692fecee/v3/translate?version=2018-05-01"
headers = {"Content-Type": "application/json"}
auth = ("apikey", api_key)

In [27]:
data = {"text":[to_translate],"model_id":"en-es"}

In [28]:
json.dumps(data)

'{"text": ["hello"], "model_id": "en-es"}'

In [29]:
r = requests.post(headers=headers, auth=auth, url=url, data=json.dumps(data))

In [31]:
r.json()

{'translations': [{'translation': 'hola'}],
 'word_count': 1,
 'character_count': 5}

In [47]:
def translate_pt_en(en_word):
    url = "https://api.eu-de.language-translator.watson.cloud.ibm.com/instances/58781bbf-1a05-4c1d-a4ee-338e692fecee/v3/translate?version=2018-05-01"
    headers = {"Content-Type": "application/json"}
    auth = ("apikey", api_key)
    data = {"text":[en_word],"model_id":"pt-en"}
    r = requests.post(headers=headers, auth=auth, url=url, data=json.dumps(data))
    return r.json()["translations"][0]["translation"]

In [48]:
translate_pt_en("bebidas")

'drinks'

In [49]:
translate_pt_en("esporte")

'sport'

In [50]:
translate_pt_en("mesa")

'table'

In [51]:
translate_pt_en("cama")

'bed'

In [58]:
def translate_pt_en(list_of_words):
    url = "https://api.eu-de.language-translator.watson.cloud.ibm.com/instances/58781bbf-1a05-4c1d-a4ee-338e692fecee/v3/translate?version=2018-05-01"
    headers = {"Content-Type": "application/json"}
    auth = ("apikey", api_key)
    data = {"text": list_of_words,"model_id": "pt-en"}
    r = requests.post(headers=headers, auth=auth, url=url, data=json.dumps(data))
    return [transl["translation"] for transl in r.json()["translations"]]

In [59]:
translate_pt_en(["esporte", "mesa", "cama"])

['sport', 'table', 'bed']

In [60]:
translate_pt_en(["copo"])

['cup']

# Steps:

- 1. Get a list of all unique product category names
- 2. clean the strings (if necessary)
- 3. create a list with all names that we want to translate
- 4. use our function to have this list translated
- 5. create a mapping table as a dataframe

## 1. Get a list of all unique product category names

In [64]:
conn = pymysql.connect(host="localhost",
                       port=3306,
                       user="ironhack",
                       database="olist",
                       passwd=getpass.getpass())

········


In [66]:
product_categories = pd.read_sql("SELECT DISTINCT product_category_name FROM products WHERE product_category_name IS NOT NULL;", conn)

In [81]:
products_to_translate = product_categories["product_category_name"].str.replace("_", " ").to_list()

1., 2. and 3. done

## 4. Translate

In [83]:
translated = translate_pt_en(products_to_translate)

## 5. Create a translation dataframe

In [93]:
translated = [word.replace(" ", "_") for word in translated]

In [94]:
product_category_name_translations = product_categories.assign(product_category_name_en=translated)

In [95]:
product_category_name_translations

,product_category_name,product_category_name_en
0,perfumaria,perfumery
1,artes,arts
2,esporte_lazer,sport_leisure
3,bebes,babies
4,utilidades_domesticas,domestical_utilities
...,...,...
68,casa_conforto_2,house_comfort_2
69,portateis_cozinha_e_preparadores_de_alimentos,portable_cooking_and_food_preparers
70,seguros_e_servicos,safe_and_servic
71,moveis_colchao_e_estofado,movable_collchao_and_upholstery
